In [2]:
import cdsapi
import xarray as xr
import pygrib
import numpy as np
import pandas as pd

import requests
import os
import xarray as xr
#What a permaculture project need to know when they start
import time
from geopy.geocoders import Nominatim


In [6]:

# select the years you want to download:
start_year = 1992
end_year = 2022
year_range = [i for i in range(start_year, end_year + 1)]
print(year_range)



c = cdsapi.Client()
try:
    data = c.retrieve("reanalysis-era5-single-levels-monthly-means",
    {"format": "grib",
     "product_type": "monthly_averaged_reanalysis_by_hour_of_day",
     "variable": ['10m_u_component_of_wind', '10m_v_component_of_wind', 
                '2m_temperature',
                'snowfall', 
                'soil_temperature_level_1', 
                'soil_type',
                'total_cloud_cover', 
                'total_precipitation',
                ],
    "area": [72,-11, 33, 40],  # North, West, South, East. 
    "year": year_range,
    "month": ['01', '02', '03',
           '04', '05', '06',
           '07', '08', '09',
           '10', '11', '12'],
    "time": ["00:00","01:00","02:00","03:00","04:00","05:00",
             "06:00","07:00","08:00","09:00","10:00","11:00",
             "12:00", "13:00","14:00","15:00","16:00","17:00",
             "18:00","19:00","20:00","21:00","22:00","23:00"]
    })

    # Get the location of the file to download
    url = data.location

    # Download the file
    response = requests.get(url)

    # Check if the request was successful
    response.raise_for_status()

except requests.exceptions.HTTPError as errh:
    print ("HTTP Error:",errh)
except requests.exceptions.ConnectionError as errc:
    print ("Error Connecting:",errc)
except requests.exceptions.Timeout as errt:
    print ("Timeout Error:",errt)
except requests.exceptions.RequestException as err:
    print ("Something went wrong with the request:",err)

else:
    # If the request was successful, write the file
    filename = 'my_app/data/download.grib'
    with open(filename, 'wb') as f:
        f.write(response.content)

    # Print the location where the file is saved


[1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]


2023-12-07 11:07:00,757 INFO Welcome to the CDS
2023-12-07 11:07:00,757 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2023-12-07 11:07:00,842 INFO Request is queued


KeyboardInterrupt: 

In [3]:
print(f"File saved at: {os.path.abspath(filename)}")

File saved at: /Users/giacomo/Documents/projects/permaculture-climate/download.grib


In [30]:
filename = "my_app/data/download.grib"

# List of variables to load
variables = ['2t','10v','10u','tp','tcc']

# Dictionary to hold the datasets
datasets = {}
lat = 40
lon = -2
# Open the GRIB file for each variable using the short name parameter
for var in variables:
    ds = xr.open_dataset(filename, engine='cfgrib', 
                         backend_kwargs={'filter_by_keys': {'shortName': var}})
    print(ds)
    ds = ds.sel(latitude=slice(lat + 1, lat - 1), longitude=slice(lon - 1, lon + 1))
    datasets[var] = ds


<xarray.Dataset>
Dimensions:     (time: 8928, latitude: 157, longitude: 205)
Coordinates:
    number      int64 ...
  * time        (time) datetime64[ns] 1992-01-01 ... 2022-12-01T23:00:00
    step        timedelta64[ns] ...
    surface     float64 ...
  * latitude    (latitude) float64 72.0 71.75 71.5 71.25 ... 33.5 33.25 33.0
  * longitude   (longitude) float64 -11.0 -10.75 -10.5 ... 39.5 39.75 40.0
    valid_time  (time) datetime64[ns] ...
Data variables:
    t2m         (time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2023-12-06T14:11 GRIB to CDM+CF via cfgrib-0.9.1...
<xarray.Dataset>
Dimensions:     (time: 8928, latitude: 157, longitude: 205)
Coordinates:
    nu

In [7]:
start_year = 2015
end_year = 2045
year_range_forecast = [str(i) for i in range(start_year, end_year + 1)]
print(year_range)

dataset_variables = ['air_temperature', 'eastward_near_surface_wind', 'northward_near_surface_wind', 'precipitation']
for variable in dataset_variables:
    print('download_' + variable + '.grib')

# Use geopy to get the latitude and longitude of the city
geolocator = Nominatim(user_agent="permaculture-climate")
location = geolocator.geocode("Puebla de don Fadrique, Spain")
# Add a delay between requests
time.sleep(1)



[1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
download_air_temperature.grib
download_eastward_near_surface_wind.grib
download_northward_near_surface_wind.grib
download_precipitation.grib


In [11]:
c = cdsapi.Client()

dataset_variables = ['air_temperature', 'eastward_near_surface_wind', 'northward_near_surface_wind', 'precipitation']
for variable in dataset_variables:
    #create a loop as air temperature also needs 'level' while the others dont
    if variable == 'air_temperature':            
        data = c.retrieve(
            'projections-cmip6',
            {
                'format': 'zip',
                'temporal_resolution': 'monthly',
                'variable': variable,
                'experiment': 'ssp2_4_5',
                'model': 'cmcc_esm2',
                'area':[location.latitude + 1, 
                    location.longitude - 1, 
                    location.latitude - 1, 
                    location.longitude + 1],  # North, West, South, East. 
                'month': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                ],
                'year': year_range_forecast,
                'level': '1000',
            },
            variable + '.zip')
        url = data.location

        # Download the file
        response = requests.get(url)

       # Check if the request was successful
        response.raise_for_status()
    else:         
        data = c.retrieve(
            'projections-cmip6',
            {
                'format': 'zip',
                'temporal_resolution': 'monthly',
                'variable': variable,
                'experiment': 'ssp2_4_5',
                'model': 'cmcc_esm2',
                'area':[location.latitude + 1, 
                    location.longitude - 1, 
                    location.latitude - 1, 
                    location.longitude + 1],  # North, West, South, East. 
                'month': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                ],
                'year': year_range_forecast,
                
            },
            variable + '.zip')    
    # Get the location of the file to download
        url = data.location

        # Download the file
        response = requests.get(url)

        # Check if the request was successful
        response.raise_for_status()

    # If the request was successful, write the file
    filename = 'download_' + variable + '.grib'
    with open(filename, 'wb') as f:
        f.write(response.content)

        # Print the location where the file is saved
        print(f"File saved at: {os.path.abspath(filename)}")
print('Downloads completed')


2023-12-07 11:13:49,381 INFO Welcome to the CDS
2023-12-07 11:13:49,382 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
2023-12-07 11:13:49,571 INFO Request is completed
2023-12-07 11:13:49,572 INFO Downloading https://download-0012-clone.copernicus-climate.eu/cache-compute-0012/cache/data5/adaptor.esgf_wps.retrieve-1701939571.9456537-19457-5-cd28dd71-1fc9-405d-93cc-4971880ce192.zip to air_temperature.zip (210.7K)
2023-12-07 11:13:49,829 INFO Download rate 821.6K/s
2023-12-07 11:13:50,284 INFO Welcome to the CDS
2023-12-07 11:13:50,285 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
2023-12-07 11:13:50,421 INFO Downloading https://download-0009-clone.copernicus-climate.eu/cache-compute-0009/cache/data7/adaptor.esgf_wps.retrieve-1701939598.9053617-30161-14-700b752d-bc8b-4adb-8b6b-9f47b1473550.zip to eastward_near_surface_wind.zip (281.8K)


File saved at: /Users/giacomo/Documents/projects/permaculture-climate/download_air_temperature.grib


2023-12-07 11:13:50,749 INFO Download rate 860K/s 
2023-12-07 11:13:51,191 INFO Welcome to the CDS
2023-12-07 11:13:51,192 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
2023-12-07 11:13:51,294 INFO Downloading https://download-0021.copernicus-climate.eu/cache-compute-0021/cache/data6/adaptor.esgf_wps.retrieve-1701939621.186981-31718-7-8dba7b2d-2a83-400b-bad9-f93c8afb25aa.zip to northward_near_surface_wind.zip (259.2K)


File saved at: /Users/giacomo/Documents/projects/permaculture-climate/download_eastward_near_surface_wind.grib


2023-12-07 11:13:51,618 INFO Download rate 800.1K/s
2023-12-07 11:13:52,052 INFO Welcome to the CDS
2023-12-07 11:13:52,053 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
2023-12-07 11:13:52,184 INFO Downloading https://download-0011-clone.copernicus-climate.eu/cache-compute-0011/cache/data9/adaptor.esgf_wps.retrieve-1701687663.886994-10646-15-501ccc0a-32ad-43f1-8704-7cc6b52b7507.zip to precipitation.zip (277.4K)


File saved at: /Users/giacomo/Documents/projects/permaculture-climate/download_northward_near_surface_wind.grib


2023-12-07 11:13:52,546 INFO Download rate 769.5K/s


File saved at: /Users/giacomo/Documents/projects/permaculture-climate/download_precipitation.grib
Downloads completed
